In [1]:
from preprocess import load_dataset

data = load_dataset()

提取特征

In [2]:
from feature import extract_user_feature, extract_features

user_features = extract_user_feature(data['train'], data['all_uid'])
features_train = extract_features(data['train'], user_features)
features_valid = extract_features(data['valid'], user_features)
features_test = extract_features(data['test'], user_features)
print('feature dim:', features_train.shape[1])

User Histories: 100%|██████████| 1044681/1044681 [00:25<00:00, 40588.49it/s]


feature dim: 20


In [16]:
import numpy as np
import pandas as pd

# 互动数量取对数
def extract_log_scale_targets(df: pd.DataFrame):
    output_columns = ['like_count', 'forward_count', 'comment_count']
    targets = []
    for col in output_columns:
        targets.append(np.log( df[col].to_numpy() + 1 ))
    return np.column_stack(targets)

targets_train = extract_log_scale_targets(data['train'])
targets_valid = extract_log_scale_targets(data['valid'])

线性回归 - 拟合+预测

In [4]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(features_train, targets_train)
predicts = model.predict(features_valid)
predicts.shape

(184937, 3)

In [17]:
from metric import compute_metrics

# 计算指标时恢复对数
def exp_interactions(x: np.ndarray) -> np.ndarray:
    return np.rint(np.exp(x) - 1).astype(int)
targets_valid = exp_interactions(targets_valid)
compute_metrics(exp_interactions(predicts), targets_valid)

{'score': 0.2558869537685386}

In [21]:
test_set_predicts = exp_interactions(model.predict(features_test))
results = []
for i in range(len(data['test'])):
    results.append(
        "{}\t{}\t{},{},{}\n".format(
            data['test'].loc[i, 'uid'],
            data['test'].loc[i, 'mid'],
            test_set_predicts[i, 1],
            test_set_predicts[i, 2],
            test_set_predicts[i, 0]
        )
    )
with open("submission.txt", 'w') as f:
    f.writelines(results)